<a href="https://colab.research.google.com/github/bwmelon97/fast-api-colab-test/blob/main/FastAPI_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the dependencies

In [13]:
!pip install fastapi nest-asyncio pyngrok uvicorn python-multipart

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31678 sha256=dfdc1e334f862b32ee606959d4dfa90378a1a6910800dbd878ebc3bfa73ba92b
  Stored in directory: /root/.cache/pip/wheels/2c/41/7c/bfd1c180534ffdcc0972f78c5758f89881602175d48a8bcd2c
Successfully built python-multipart


GPU 할당?

In [8]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
for i in range(len(gpus)): #모든 gpu에 동일한 양 할당하기
	tf.config.experimental.set_memory_growth(gpus[i], True)

Model load

In [10]:
import pickle
#모델 로드
pickle_in = open("/content/sample_data/model_220915.pkl","rb")
model = pickle.load(pickle_in)

Create App

In [25]:
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/')
async def root():
    return {'hello': 'world'}

@app.get('/who')
async def root():
    return {'are': 'you'}

@app.post('/how-old')
async def root(age: int):
    return {'age': age}

@app.post('/') #데이터 업로드 및 현재 실행 경로에 저장
async def root(file: UploadFile = File(...)) :
    with open(f'{file.filename}', "wb") as buffer :
        shutil.copyfileobj(file.file, buffer)
    print(type(file))
    return{"file_name" : file.filename}

#여기서 문제는 file이 <class 'starlette.datastructures.UploadFile'>인데 우리는 numpy로 바꾸어줘야함!
def load_image_into_numpy_array(data): #!!
    return np.array(Image.open(BytesIO(data)))

@app.post('/predict') #파이프라인, image를 함수안에넣는법!!
#가능한 올리자마자 바로 넣는(굳이 저장안하고.. -> 매개변수로 연결)
async def predict_num(img: UploadFile = File(...)):
    print('hello world')
    img = load_image_into_numpy_array(await img.read())
    img = np.resize(img,(28,28))
    img = np.expand_dims(img,0) #1차원의 모델은 keras에 의해 predict가 불가능해서 2차원으로 만들어주는 작업
    predictions=model.predict(img)
    best = np.argmax(predictions)
    return { "Answer" : int(best) } #fastapi는 numpy를 return 못한다!

Import server packages

In [15]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

Run the server application

In [26]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://661a-34-125-175-47.ngrok.io


INFO:     Started server process [55]
INFO:uvicorn.error:Started server process [55]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     114.70.4.158:0 - "GET / HTTP/1.1" 200 OK
INFO:     114.70.4.158:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     54.86.50.139:0 - "POST /how-old HTTP/1.1" 422 Unprocessable Entity
INFO:     54.86.50.139:0 - "POST /how-old HTTP/1.1" 422 Unprocessable Entity
INFO:     54.86.50.139:0 - "POST /how-old HTTP/1.1" 422 Unprocessable Entity


INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [55]
INFO:uvicorn.error:Finished server process [55]
